# Problem 4.b. - Job Shop Problem

In [1]:
import pandas as pd
import os
from __future__ import print_function
import collections
# Import Python wrapper for or-tools CP-SAT solver.
from ortools.sat.python import cp_model
os.chdir(r"C:\Users\User\Desktop\BUS 730 - Prescriptive Modeling & Optimization for BA\Assignments\HW3_Integer (MIP) Optimization") 
%pprint

Pretty printing has been turned OFF


In [2]:
# reading in JobShop data
JobShopdata = pd.read_excel(open('P4_JobShopData.xlsx','rb'), sheet_name='Sheet1') 
JobShopdata

,Job,Order,Machine Needed,Processing Time
0,0,1,5,22
1,0,2,4,29
2,0,3,2,11
3,0,4,1,20
4,1,1,6,36
...,...,...,...,...
69,18,4,1,19
70,18,5,0,5
71,19,1,4,25
72,19,2,5,1


In [3]:
# converting JobShop data to a list of lists of task tuples (machine_id, processing_time)

JobShopdict = collections.defaultdict(list)  # create a dictionary to store data for each job

for k in range(len(JobShopdata)):
    rowdata = JobShopdata.iloc[k].values.tolist()  # each row in JobShop data
    job_id = rowdata[0]
    machine_id = rowdata[2]
    processing_time = rowdata[3] 
    tasktuple =  (machine_id, processing_time) # task = (machine_id, processing_time)
    JobShopdict[job_id].append(tasktuple)

jobs_data = list(JobShopdict.values())
jobs_data

[[(5, 22), (4, 29), (2, 11), (1, 20)], [(6, 36), (1, 20), (4, 31)], [(2, 20), (3, 20), (4, 46), (1, 20), (0, 5)], [(4, 42), (3, 16)], [(1, 11), (0, 2), (3, 3), (6, 19)], [(6, 21), (0, 4), (1, 14), (3, 19), (2, 23)], [(0, 9), (3, 29), (4, 23)], [(3, 27), (5, 39), (6, 31)], [(2, 28), (4, 47)], [(2, 30), (5, 23), (3, 29), (1, 20), (0, 4)], [(0, 8), (2, 21), (3, 4), (6, 33), (1, 18), (5, 7)], [(2, 12), (3, 30), (4, 42), (5, 36)], [(0, 1), (1, 14), (2, 18), (3, 40)], [(4, 48), (5, 1), (3, 25), (2, 15), (1, 20)], [(5, 6), (2, 11)], [(0, 10)], [(5, 14), (2, 30), (3, 10)], [(1, 19), (2, 13), (3, 40), (6, 14), (4, 14)], [(2, 22), (6, 11), (5, 33), (1, 19), (0, 5)], [(4, 25), (5, 1), (6, 13)]]

In [4]:
def MinimalJobshopSat():
    """Minimal jobshop problem."""
    # Create the model.
    model = cp_model.CpModel()
    
    # The number of machines
    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)

    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(
                start=start_var, end=end_var, interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.Add(all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [all_tasks[job_id, len(job) - 1].end 
                                   for job_id, job in enumerate(jobs_data)])
    model.Minimize(obj_var)

    # Solve model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(
                        start=solver.Value(all_tasks[job_id, task_id].start),
                        job=job_id,
                        index=task_id,
                        duration=task[1]))

        # Create per machine output lines.
        output = ''
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                name = 'job_%i_%i' % (assigned_task.job, assigned_task.index+1)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-10s' % name

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = '[%i,%i]' % (start, start + duration)
                # Add spaces to output to align columns.
                sol_line += '%-10s' % sol_tmp
                
            sol_line += '\n\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print('Optimal Schedule Length: %i\n' % solver.ObjectiveValue())
        print('Optimal Schedule (job_jobID_order) and Task Time Intervals for Each Machine: \n')
        print(output)

MinimalJobshopSat()

Optimal Schedule Length: 347

Optimal Schedule (job_jobID_order) and Task Time Intervals for Each Machine: 

Machine 0: job_6_1   job_12_1  job_4_2   job_10_1  job_15_1  job_5_2   job_2_5   job_9_5   job_18_5  
           [0,9]     [9,10]    [11,13]   [13,21]   [21,31]   [76,80]   [114,119] [237,241] [306,311] 

Machine 1: job_4_1   job_12_2  job_1_2   job_17_1  job_5_3   job_2_4   job_0_4   job_10_5  job_9_4   job_18_4  job_13_5  
           [0,11]    [11,25]   [36,56]   [56,75]   [80,94]   [94,114]  [149,169] [199,217] [217,237] [287,306] [324,344] 

Machine 2: job_2_1   job_8_1   job_9_1   job_10_2  job_11_1  job_5_5   job_0_3   job_12_3  job_14_2  job_16_2  job_17_2  job_18_1  job_13_4  
           [0,20]    [20,48]   [48,78]   [78,99]   [99,111]  [115,138] [138,149] [149,167] [167,178] [178,208] [208,221] [221,243] [309,324] 

Machine 3: job_4_3   job_2_2   job_6_2   job_7_1   job_5_4   job_10_3  job_11_2  job_3_2   job_9_3   job_12_4  job_16_3  job_17_3  job_13_3  
           [13

### Interpretation of the solution

The problem is to schedule a sequence of tasks for several jobs on specific machines so as to minimize the length of the schedule — the time it takes for all the jobs to be completed. In this case, the optimal schedule length for completing all jobs is 347, when the Job1-Order3 task ends.

The optimal schedule and task time intervals for each machine are shown in above solution output. The tasks (job_jobID_order) for each machine are displayed in the processing order, and task time intervals are shown in brackets under the tasks.